In [1]:
import fitz
import re
from pdfminer.high_level import extract_pages
from pdfminer.layout import *
import textwrap
import math
import json
import sys

pdf_path = "civil-code.pdf"
#starting_words = ["Titre", "Title", "Chapitre", "Chapter", "Section", "Sous-section","Sub-section", "Paragraphe","Paragraph" , "Livre", "Book" ]

In [47]:
doc = fitz.open(pdf_path)
font = {}
i = 0

file = open("font.txt", 'w', encoding='utf-8') 

for page in doc:
    blocks =  page.get_text("dict", flags=11)["blocks"]
    for block in blocks:
        for l in block["lines"]:  # iterate through the text lines
            for s in l["spans"]:  # iterate through the text spans
                font[i] = [s["text"], s["font"], s["size"], s["bbox"]]
                i += 1

doc.close()

In [40]:
font

{0: ['THAT KIND OF GUY',
  'JosefinSans',
  21.2592830657959,
  (202.26751708984375,
   81.56968688964844,
   392.718017578125,
   102.82896423339844)],
 1: ['STEPHANIE ARCHER',
  'JosefinSans',
  21.2592830657959,
  (197.21392822265625, 80.81925964355469, 397.783203125, 102.07854461669922)],
 2: ['This is a work of fiction. Names, characters, places, and incidents are products of',
  'TimesNewRomanPSMT',
  11.25623893737793,
  (108.01996612548828,
   79.2289810180664,
   474.0730285644531,
   91.69438934326172)],
 3: ['the author’s imagination or are used fictitiously and are not to be construed as real.',
  'TimesNewRomanPSMT',
  11.25623893737793,
  (108.01996612548828,
   91.98604583740234,
   480.7625427246094,
   104.45145416259766)],
 4: ['Any resemblance to actual events, locales, organizations, or persons, living or dead,',
  'TimesNewRomanPSMT',
  11.25623893737793,
  (108.01996612548828,
   104.74312591552734,
   485.4222412109375,
   117.20853424072266)],
 5: ['is entirely 

In [49]:
with open("font.txt", "w", encoding='utf-8') as f:
    f.write(str(font))

f.close()

In [5]:
#def comp_fontsize(dictionary, string):
#    keys = list(dictionary.keys())
#    
#    for i in range(len(keys) - 1):
#        curr_key = keys[i]
#        next_key = keys[i+1]
#    
#        if dictionary[curr_key][0] == string and dictionary[curr_key][2] == dictionary[next_key][2]:
#            # Continue iterating if font sizes are the same
#            continue
#        elif dictionary[curr_key][0] == string:
#            return False
#    
#    return True
#

In [2]:
def get_footers(fitz_doc):
    footers = []
    for page in fitz_doc:
        data = page.get_text("blocks")
        h = page.rect.height

        for l in data:
            if l[1] > 0.90*h:
                text = l[4]
                footers.append(text[:-1]) # remove the newline character at end of footer

    return footers

def extract_sentences_with_starting_words(pdf_path, starting_words):
    doc = fitz.open(pdf_path)
    word_sentences = [[] for _ in range(len(starting_words))]
    current_word = None
    previous_line_empty = False
    previous_line_word = None

    # Get footers to check later if line is a footer
    footers = get_footers(doc)

    for page in doc:
        text = page.get_text()
        lines = text.splitlines()
        for line in lines:
            if line.strip():  # Check if the line is not empty
                # Check if the line starts with any of the starting words
                for i, word in enumerate(starting_words):
                    if line.lower().startswith(word.lower() + ' ') and line[0].isupper():
                        current_word = i
                        word_sentences[current_word].append(line.strip())
                        previous_line_word = i
                        break
                else:
                    # Append the line to the current sentence if a starting word is detected in the previous line
                    if current_word is not None and line not in footers:
                        word_sentences[current_word][-1] += ' ' + line.strip()
                        previous_line_word = current_word
            elif previous_line_word is not None and not previous_line_empty:
                # End the sentence if the current line is empty and the previous line wasn't empty
                current_word = None
                previous_line_word = None

            previous_line_empty = not line.strip()  # Update the flag for the previous line

    doc.close()
    return word_sentences

In [11]:
def concat_strings(input_list):
    result_list = []
    temp_string = ""
    prev_attributes = None

    for item in input_list:
        text, font, size, float_val = item
        attributes = (font, size, float_val)

        if attributes == prev_attributes: # and not re.search(r'Bold', font, re.IGNORECASE):
            if (text[0].isupper() and re.search(r'Bold', font, re.IGNORECASE)) or text[0].isupper():
                temp_string = text
            else:
                temp_string += " " + text
        else:
            if temp_string:
                result_list.append([temp_string.strip()] + list(prev_attributes))
            temp_string = text
            prev_attributes = attributes

    # Append the last concatenated string
    if temp_string:
        result_list.append([temp_string.strip()] + list(prev_attributes))

    return result_list


In [4]:
def extract_text_by_fontsize(pdf_url):
    extracted_text = ""
    curr_font = None
    curr_size = None
    font_attr = []

    for page_layout in extract_pages(pdf_url):
        for element in page_layout:
            if isinstance(element, LTTextBox):
                for line in element:
                    thisline = []
                    if isinstance(line, LTTextLine):
                        for char in line:
                            if isinstance(char, LTChar):
                                ft = char.fontname
                                sz = math.ceil(char.size)
                                x = char.bbox[0]
    
                                #if ft != curr_font or sz != curr_size:
    
                                l = line.get_text()
                                thisline.append(l[:-1])
                                thisline.append(ft)
                                thisline.append(sz)
                                thisline.append(x)
                                font_attr.append(thisline)        
                            break

    result_list = concat_strings(font_attr)
    return result_list



In [5]:
def classify_text(font_name, text, font_size, x):
  # Classification rules here based on font_name, font_size, and y (position)
  if (re.search(r'Bold', font_name, re.IGNORECASE) and font_size > 14) or font_size > 14 or (font_size > 14 and x > 125) or text.isupper():  # Threshold based on PDF
    return "Titre"
  elif re.search(r'Bold', font_name, re.IGNORECASE) and font_size > 13:
    return "Section"
  elif font_size > 8:
    return "Paragraphe"
  else:
    return "Footer"


def process_pdf(pdf_text):
  l2 = []
  for line_attr in pdf_text:
    
    text = line_attr[0]
    font_name = line_attr[1]
    font_size = line_attr[2]
    x = line_attr[3]
    category = classify_text(font_name, text, font_size, x)
    # Do something with the classified text and category
    #print(f"{text}, Category: {category}")
    map = (category, text)
    l2.append(map)
  return l2

In [6]:
def comp(l1, l2):
    # Convert list1 to a set of sentences for efficient comparison
    set1 = set([(category, sentence) for category, sentence in l1])

    combined_sentences = l1[:]
    
    temp = []
    for c, s in l1:
        temp.append(s)
    
    for category, sentence in l2:
    
        if sentence not in temp:
            combined_sentences.append((category, sentence))

    def sort_key(item):
    # Get the first key in the dictionary and return it
        return next(iter(item))

    # Sort the list of dictionaries
    sorted_data = sorted(combined_sentences, key=sort_key)
    
    return sorted_data    


In [7]:
def make_map(suchi):
    naksha = {}
    for i, (c, s)  in enumerate(suchi):
        naksha[i] = {"Category": c, "Text":s}

    unique_map = {}
    for key, value in naksha.items():
        if value not in unique_map.values():
            unique_map[key] = value

    res = {}
    i = 0
    for key, value in unique_map.items():
        res[i] = value
        i += 1
    
    return res


In [10]:
if __name__ == "__main__":
    #if len(sys.argv) != 2:
    #    print("Command: python main.py <pdf path>")
    #    sys.exit(1)
    #pdf_path = "C:\\Users\\AKSHITA RAI\\Downloads\\Catalogue des cours Formation d'ingénieur centralien - 2022-11-29 (1).pdf"
    pdf_path = "civil-code.pdf"

    starting_words = ["Titre", "Title", "Chapitre", "Chapter", "Section", "Sous-section","Sub-section", "Paragraphe","Paragraph", "Article", "Livre", "Book" ]
    
    # Extract sentences with specified starting words
    word_sentences = extract_sentences_with_starting_words(pdf_path, starting_words)

    l1 = []
    for i, word in enumerate(starting_words):
        for sentence in word_sentences[i]:
            map = (word, sentence)
            l1.append(map)

    pdf_text = extract_text_by_fontsize(pdf_path)
    l2 = process_pdf(pdf_text)

    compare = comp(l1, l2)
    naksha = make_map(compare)

    outfile = open("output_uni.json", "w", encoding = 'utf-8') 
    json.dump(naksha, outfile,indent=2, ensure_ascii=False)
    outfile.close()